In [ ]:
import logging

from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.datasets import imdb
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Conv1D, MaxPooling1D, Activation
from keras.models import Sequential
from keras.preprocessing import sequence

# Polyaxon
from polyaxon_client.tracking import Experiment
from polyaxon_client.tracking.contrib.keras import PolyaxonKeras

In [ ]:
logger = logging.getLogger('bidir-lstm')

OPTIMIZERS = {
    'adam': optimizers.Adam,
    'rmsprop': optimizers.RMSprop,
    'sgd': optimizers.SGD,
}

In [ ]:
def transform_data(x_train, y_train, x_test, y_test, maxlen):
    x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
    x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
    return x_train, y_train, x_test, y_test


def train(experiment,
          max_features,
          maxlen,
          embedding_size,
          kernel_size,
          optimizer,
          filters, pool_size, lstm_output_size,
          log_learning_rate,
          batch_size,
          epochs):
    model = Sequential()
    model.add(Embedding(max_features, embedding_size, input_length=maxlen))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters,
                     kernel_size,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(LSTM(lstm_output_size))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(OPTIMIZERS[optimizer](lr=10 ** log_learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              callbacks=[PolyaxonKeras(experiment=experiment)])

    score, accuracy = model.evaluate(x_test, y_test, batch_size=batch_size)
    return score, accuracy

In [ ]:
max_features_param=20000
skip_top_param=50
maxlen_param=100
epochs_param=1
embedding_size_param=128
pool_size_param=4
kernel_size_param=5
filters_param=64
lstm_output_size_param=70
batch_size_param=32
optimizer_param='adam'
log_learning_rate_param=-3

In [ ]:
experiment = Experiment('cnn-lstm')
experiment.create(framework='keras', tags=['examples'])
experiment.log_params(max_features=max_features_param,
                      skip_top=skip_top_param,
                      maxlen=maxlen_param,
                      epochs=epochs_param,
                      embedding_size=embedding_size_param,
                      pool_size=pool_size_param,
                      kernel_size=kernel_size_param,
                      filters=filters_param,
                      lstm_output_size=lstm_output_size_param,
                      batch_size=batch_size_param,
                      optimizer=optimizer_param,
                      log_learning_rate=log_learning_rate_param)

logger.info('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features_param,
                                                      skip_top=skip_top_param)

logger.info('train sequences %s', len(x_train))
logger.info('test sequences %s', len(x_test))

# Polyaxon
experiment.log_data_ref(data=x_train, data_name='x_train')
experiment.log_data_ref(data=y_train, data_name='y_train')
experiment.log_data_ref(data=x_test, data_name='x_test')
experiment.log_data_ref(data=y_test, data_name='y_test')

logger.info('Transforming data...')
x_train, y_train, x_test, y_test = transform_data(x_train,
                                                  y_train,
                                                  x_test,
                                                  y_test,
                                                  maxlen_param)

logger.info('Training...')
score, accuracy = train(experiment=experiment,
                        max_features=max_features_param,
                        maxlen=maxlen_param,
                        epochs=epochs_param,
                        embedding_size=embedding_size_param,
                        pool_size=pool_size_param,
                        kernel_size=kernel_size_param,
                        filters=filters_param,
                        lstm_output_size=lstm_output_size_param,
                        batch_size=batch_size_param,
                        optimizer=optimizer_param,
                        log_learning_rate=log_learning_rate_param)

# Polyaxon
experiment.log_metrics(score=score, accuracy=accuracy)

logger.info('Test score: %s', score)
logger.info('Test accuracy: %s', accuracy)